In [1]:
import pandas as pd
from collections import defaultdict

In [54]:
suspensions = pd.read_csv('suspensions.csv')

In [55]:
def filter_olympic(dataframe, target_column = 'school', false_numerics=[], unique_identifiers=[]):
    dataframe = dataframe.copy()
    olympic = dataframe.loc[dataframe[target_column].str.contains('Olympic')].copy()
    for column in false_numerics:
        olympic[column] = olympic[column].apply(lambda x: str(x))
    olympic['key'] = ''
    for identifier in unique_identifiers:
        olympic['key'] =  olympic['key'] + olympic[identifier]
    olympic.set_index('key', inplace=True)
    new_olympic = defaultdict(dict)
    for label, content in olympic.items():
        i = 0
        for item in content:
            key = content.index[i]
            if type(item) == int:
                if label in new_olympic[key].keys():
                    new_olympic[key][label] += item
                else:
                    new_olympic[key][label] = item
            else:
                new_olympic[key][label] = item
                new_olympic[key][target_column] = "Olympic High"
            i += 1
    dataframe_out = dataframe[dataframe[target_column].str.contains('Olympic') == False]
    dataframe_out = dataframe_out.append(list(new_olympic.values()))
    return dataframe_out

In [56]:
def filter_m_f(dataframe, target_column = 'school', false_numerics=[], unique_identifiers=[], filter_criteria = []):
    dataframe = dataframe.copy()
    olympic = pd.DataFrame()
    for filtr in filter_criteria:
        olympic = olympic.append(dataframe.loc[dataframe[target_column].str.contains(filtr)])
    for column in false_numerics:
        olympic[column] = olympic[column].apply(lambda x: str(x))
    olympic['key'] = ''
    for identifier in unique_identifiers:
        olympic['key'] =  olympic['key'] + olympic[identifier]
    olympic.set_index('key', inplace=True)
    new_olympic = defaultdict(dict)
    for label, content in olympic.items():
        i = 0
        for item in content:
            key = content.index[i]
            if type(item) == int:
                if label in new_olympic[key].keys():
                    new_olympic[key][label] += item
                else:
                    new_olympic[key][label] = item
            else:
                new_olympic[key][label] = item
            i += 1
    dataframe_out = pd.DataFrame()
    dataframe_out = dataframe_out.append(list(new_olympic.values()))
    dataframe_out.drop('sex', axis=1, inplace=True)
    return dataframe_out

In [57]:
suspensions_condensed = filter_olympic(suspensions, 'school', ['year'], ['year', 'sex', 'category'])

In [58]:
m_f = filter_m_f(suspensions_condensed, 'sex', ['year', 'id'], ['id', 'year', 'school', 'category'], ['M', 'F'])

In [64]:
m_f.to_csv('suspensions_final.csv', index=False)

In [60]:
absent = pd.read_csv('absenteeism.csv')

In [61]:
absenteeism_condensed = filter_olympic(suspensions, 'school', ['year'], ['year', 'sex', 'category'])

In [62]:
absent_gender_combined = filter_m_f(suspensions_condensed, 'sex', ['year', 'id'], ['id', 'year', 'school', 'category'], ['M', 'F'])

In [63]:
absent_gender_combined.to_csv('absenteeism_final.csv', index=False)

In [2]:
melt_amendment = pd.read_csv('absenteeism_final.csv')

In [3]:
melt_amendment.head()

,lea_state,lea,school_state,school,id,year,category,american_indian,asian,pacific_island,hispanic,black,white,two_or_more,total,swd_idea,swd_401,el
0,NC,Charlotte-Mecklenburg Schools,NC,Ardrey Kell High,600302,2015,School days missed due to out-of-school suspen...,0,2,0,36,391,99,12,540,23,0,0
1,NC,Charlotte-Mecklenburg Schools,NC,Ardrey Kell High,600302,2015,Total enrollment,4,454,4,235,355,1759,70,2881,167,64,73
2,NC,Charlotte-Mecklenburg Schools,NC,Ardrey Kell High,600302,2017,School days missed due to out-of-school suspen...,0,14,0,22,43,160,1,240,9,3,8
3,NC,Charlotte-Mecklenburg Schools,NC,Butler High,600361,2015,School days missed due to out-of-school suspen...,0,1,0,280,528,179,26,1014,90,48,117
4,NC,Charlotte-Mecklenburg Schools,NC,Butler High,600361,2015,Total enrollment,10,88,2,409,676,853,88,2126,170,64,103


In [4]:
melt_amendment.columns

Index(['lea_state', 'lea', 'school_state', 'school', 'id', 'year', 'category',
       'american_indian', 'asian', 'pacific_island', 'hispanic', 'black',
       'white', 'two_or_more', 'total', 'swd_idea', 'swd_401', 'el'],
      dtype='object')

In [14]:
ids = pd.read_csv('final_school_list.csv')['school_id']

In [15]:
final_schools = melt_amendment.loc[melt_amendment['id'].isin(ids)]

In [16]:
final_schools = pd.melt(final_schools, id_vars=['lea_state', 'lea', 'school_state', 'school', 'id', 'year', 'category'], value_vars=['american_indian', 'asian', 'pacific_island', 'hispanic', 'black',
       'white', 'two_or_more', 'total', 'swd_idea', 'swd_401', 'el'])

In [17]:
final_schools.head()

,lea_state,lea,school_state,school,id,year,category,variable,value
0,NC,Charlotte-Mecklenburg Schools,NC,Ardrey Kell High,600302,2015,School days missed due to out-of-school suspen...,american_indian,0
1,NC,Charlotte-Mecklenburg Schools,NC,Ardrey Kell High,600302,2015,Total enrollment,american_indian,4
2,NC,Charlotte-Mecklenburg Schools,NC,Ardrey Kell High,600302,2017,School days missed due to out-of-school suspen...,american_indian,0
3,NC,Charlotte-Mecklenburg Schools,NC,Butler High,600361,2015,School days missed due to out-of-school suspen...,american_indian,0
4,NC,Charlotte-Mecklenburg Schools,NC,Butler High,600361,2015,Total enrollment,american_indian,10


In [19]:
suspension = pd.read_csv('suspensions_final.csv')
suspension_schools = melt_amendment.loc[melt_amendment['id'].isin(ids)]
suspension_schools = pd.melt(suspension_schools, id_vars=['lea_state', 'lea', 'school_state', 'school', 'id', 'year', 'category'], value_vars=['american_indian', 'asian', 'pacific_island', 'hispanic', 'black',
       'white', 'two_or_more', 'total', 'swd_idea', 'swd_401', 'el'])

In [20]:
final_schools.to_csv('absenteeism_melt.csv')
suspension_schools.to_csv('suspension_melt.csv')